In [2]:
import numpy as np 
import pandas as pd
import re
import xgboost

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from subprocess import check_output



In [3]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [4]:
list(train.columns.values)

['Id',
 'Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40',
 'Cover_Type']

In [5]:
#separating "ID" and response variable "Cover_Type"
Id=test['Id']
y=train['Cover_Type']

In [6]:
train.describe()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
count,15120.00000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,...,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000
mean,7560.50000,2749.322553,156.676653,16.501587,227.195701,51.076521,1714.023214,212.704299,218.965608,135.091997,...,0.045635,0.040741,0.001455,0.006746,0.000661,0.002249,0.048148,0.043452,0.030357,4.000000
std,4364.91237,417.678187,110.085801,8.453927,210.075296,61.239406,1325.066358,30.561287,22.801966,45.895189,...,0.208699,0.197696,0.038118,0.081859,0.025710,0.047368,0.214086,0.203880,0.171574,2.000066
min,1.00000,1863.000000,0.000000,0.000000,0.000000,-146.000000,0.000000,0.000000,99.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3780.75000,2376.000000,65.000000,10.000000,67.000000,5.000000,764.000000,196.000000,207.000000,106.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,7560.50000,2752.000000,126.000000,15.000000,180.000000,32.000000,1316.000000,220.000000,223.000000,138.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
75%,11340.25000,3104.000000,261.000000,22.000000,330.000000,79.000000,2270.000000,235.000000,235.000000,167.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
max,15120.00000,3849.000000,360.000000,52.000000,1343.000000,554.000000,6890.000000,254.000000,254.000000,248.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000


In [7]:
train.isnull().sum()

Id                                    0
Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Wilderness_Area2                      0
Wilderness_Area3                      0
Wilderness_Area4                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0


In [8]:
train=train.drop(['Id','Cover_Type'],1)
test=test.drop(['Id'],1)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.3, random_state=42)

In [10]:
rf = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0.4, learning_rate=0.1,
       max_delta_step=0, max_depth=6, min_child_weight=7, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [11]:
rf.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.4,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=7, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=1, verbosity=1)

In [12]:
pred=rf.predict(x_test)

In [13]:
#accuracy on cv data
acc=rf.score(x_test,y_test)
print(acc)

0.8212081128747796


In [14]:
rf.fit(train,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.4,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=7, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=1, verbosity=1)

In [15]:
ct=rf.predict(test)
print(ct)

[2 2 1 ... 3 3 3]


In [16]:
output=pd.DataFrame(Id)
output['Cover_Type']=ct
output.head()

,Id,Cover_Type
0,15121,2
1,15122,2
2,15123,1
3,15124,1
4,15125,1


In [17]:
output.to_csv("output_submission.csv",index=False)

In [18]:
for p, y in zip(pred, y_test):
    print(p,"|" ,y)

6 | 6
6 | 3
1 | 1
1 | 1
3 | 3
5 | 5
1 | 1
2 | 2
6 | 6
3 | 3
5 | 5
1 | 2
3 | 3
5 | 5
5 | 5
4 | 4
2 | 2
1 | 1
2 | 2
4 | 4
2 | 1
3 | 3
2 | 1
1 | 2
2 | 1
4 | 4
6 | 3
7 | 7
4 | 4
3 | 3
7 | 7
6 | 6
5 | 5
1 | 1
6 | 6
3 | 3
4 | 4
6 | 6
5 | 5
5 | 5
3 | 3
5 | 5
6 | 6
4 | 4
3 | 3
5 | 5
1 | 2
6 | 6
2 | 2
2 | 2
4 | 4
1 | 2
3 | 3
1 | 1
3 | 3
5 | 2
6 | 6
4 | 4
1 | 1
5 | 5
5 | 5
7 | 7
5 | 5
7 | 7
2 | 2
5 | 5
5 | 5
1 | 1
6 | 3
5 | 5
6 | 6
3 | 3
5 | 5
1 | 2
2 | 6
4 | 4
2 | 2
6 | 6
3 | 3
2 | 2
7 | 7
3 | 6
1 | 1
3 | 3
2 | 1
5 | 5
6 | 6
1 | 1
5 | 6
5 | 5
6 | 6
7 | 7
7 | 7
4 | 4
4 | 3
5 | 5
5 | 5
1 | 2
3 | 4
7 | 7
1 | 1
2 | 5
3 | 3
5 | 5
7 | 1
7 | 7
6 | 6
6 | 6
4 | 4
1 | 1
1 | 1
2 | 2
1 | 1
2 | 1
1 | 1
5 | 5
7 | 7
2 | 2
2 | 2
7 | 7
7 | 1
3 | 3
5 | 5
4 | 6
2 | 2
2 | 2
2 | 1
1 | 2
6 | 6
5 | 5
3 | 3
2 | 2
2 | 1
5 | 5
3 | 3
7 | 7
5 | 6
7 | 7
7 | 7
7 | 7
6 | 5
7 | 7
3 | 5
6 | 6
3 | 5
7 | 7
7 | 7
1 | 1
5 | 1
5 | 5
7 | 7
3 | 3
3 | 3
4 | 4
1 | 1
2 | 2
5 | 5
4 | 4
3 | 3
6 | 6
5 | 5
5 | 5
5 | 5
1 | 1
5 | 5
5 | 5
4 | 

5 | 2
7 | 7
1 | 1
2 | 2
1 | 1
5 | 5
4 | 4
4 | 4
4 | 6
1 | 1
7 | 7
2 | 1
7 | 7
3 | 3
1 | 2
5 | 5
7 | 7
2 | 2
5 | 5
7 | 7
5 | 5
2 | 2
3 | 3
6 | 6
7 | 7
4 | 4
3 | 6
5 | 2
1 | 1
5 | 5
6 | 6
4 | 4
5 | 5
2 | 2
7 | 7
6 | 3
6 | 6
4 | 4
6 | 6
6 | 6
4 | 4
7 | 7
5 | 5
3 | 3
6 | 3
3 | 4
7 | 7
3 | 3
4 | 4
6 | 6
4 | 4
7 | 7
5 | 5
5 | 5
2 | 2
4 | 4
4 | 4
7 | 7
3 | 3
2 | 2
6 | 6
4 | 4
7 | 7
4 | 4
1 | 1
1 | 1
4 | 4
4 | 4
2 | 2
4 | 4
5 | 5
2 | 2
7 | 7
1 | 1
6 | 6
7 | 2
1 | 1
7 | 7
1 | 2
7 | 7
3 | 3
5 | 5
5 | 5
1 | 1
1 | 1
3 | 3
6 | 6
7 | 7
5 | 5
3 | 6
6 | 6
5 | 5
2 | 1
5 | 2
5 | 5
3 | 3
1 | 1
2 | 2
7 | 7
4 | 4
7 | 7
5 | 5
6 | 3
3 | 3
7 | 7
4 | 4
4 | 4
4 | 4
4 | 4
3 | 6
2 | 2
7 | 7
5 | 5
6 | 6
5 | 5
3 | 3
4 | 4
7 | 7
3 | 3
1 | 1
1 | 1
7 | 7
7 | 7
5 | 5
4 | 4
3 | 3
6 | 6
1 | 1
4 | 4
1 | 1
5 | 5
4 | 4
5 | 5
1 | 2
7 | 7
5 | 5
2 | 5
7 | 1
7 | 7
2 | 2
1 | 2
2 | 1
3 | 3
4 | 4
2 | 5
7 | 7
5 | 5
2 | 2
3 | 3
6 | 3
5 | 5
6 | 2
5 | 5
6 | 3
2 | 2
4 | 6
6 | 6
3 | 3
5 | 2
3 | 3
2 | 2
5 | 2
6 | 2
7 | 7
2 | 2
7 | 7
1 | 

2 | 1
3 | 3
5 | 5
6 | 6
5 | 5
5 | 5
1 | 1
6 | 6
3 | 3
2 | 1
4 | 4
2 | 2
6 | 6
2 | 2
1 | 2
1 | 1
6 | 6
7 | 7
3 | 3
2 | 2
1 | 1
2 | 2
5 | 5
5 | 5
6 | 3
6 | 3
5 | 5
4 | 4
7 | 7
3 | 3
5 | 5
5 | 3
7 | 7
4 | 4
1 | 1
4 | 4
2 | 2
5 | 5
5 | 5
2 | 2
6 | 6
3 | 3
6 | 6
1 | 1
4 | 4
3 | 3
3 | 3
2 | 1
2 | 2
6 | 3
4 | 4
4 | 4
7 | 7
4 | 4
1 | 1
7 | 7
3 | 3
7 | 1
2 | 2
3 | 6
4 | 4
4 | 6
4 | 4
6 | 6
3 | 3
7 | 7
7 | 7
5 | 5
5 | 5
7 | 7
1 | 1
7 | 7
5 | 5
1 | 7
6 | 6
4 | 4
5 | 5
5 | 5
1 | 1
4 | 4
2 | 2
6 | 6
5 | 6
5 | 5
6 | 6
3 | 3
5 | 5
6 | 6
4 | 4
1 | 1
7 | 7
4 | 6
6 | 6
2 | 2
6 | 6
5 | 5
7 | 7
6 | 3
2 | 2
5 | 5
5 | 5
2 | 1
6 | 6
3 | 3
6 | 6
1 | 1
2 | 1
4 | 4
1 | 1
7 | 7
1 | 1
4 | 4
3 | 3
4 | 4
2 | 2
5 | 3
3 | 1
4 | 4
5 | 5
1 | 1
1 | 1
6 | 6
2 | 2
1 | 1
4 | 4
6 | 6
5 | 5
2 | 2
2 | 1
3 | 3
1 | 5
5 | 5
6 | 3
4 | 4
7 | 1
7 | 7
3 | 3
5 | 5
6 | 1
4 | 4
7 | 7
7 | 7
6 | 6
6 | 3
4 | 4
3 | 3
1 | 2
3 | 3
7 | 7
6 | 6
7 | 7
1 | 1
5 | 5
2 | 2
7 | 7
1 | 1
4 | 4
5 | 5
6 | 6
5 | 1
4 | 4
2 | 2
7 | 7
2 | 1
5 | 5
6 | 6
3 | 